# **SpaceX  Falcon 9 First Stage Landing Prediction**

## **Launch Sites Locations Analysis with Folium** :

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


So, we will be performing some interactive visual analytics using `Folium` to find some other insights about launch sites and its proximities.

Let's first import required Python packages for this lab:

In [4]:
#import piplite
#await piplite.install(['folium'])
#await piplite.install(['pandas'])
!pip install folium

     ------------------------------------ 102.3/102.3 kB 586.3 kB/s eta 0:00:00


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Marking all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates.

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site compared to the dataset used within last section of EDA with SQL.

In [3]:
# Download and read the `spacex_launch_geo.csv` from IBM cloud gallery using fetch from js library

'''
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)
'''

# Download and load the same data after wrangling it in the last section

spacex_df = pd.read_csv('Data-Wrangling-Dataset.csv')

In [4]:
spacex_df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


Now, you can take a look at what are the coordinates for each site :

In [5]:
# Select relevant sub-columns: `LaunchSite`, `Latitude`, `Longitude`, `Class`
spacex_df = spacex_df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608059,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


Above coordinates are just plain numbers that can not give you any intuitive insights about where those launch sites are. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a deep orange circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a deep orange circle at NASA Johnson Space Center's coordinate with an icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

=> and you should find a small deep orange circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in dataframe `launch_sites` :

In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Latitude, Longitude) values. In addition, add Launch site name as a popup label
for label, row in launch_sites_df.iterrows():
    # Create a deep orange circle at NASA Johnson Space Center's coordinate with a popup label showing its name
    circle = folium.Circle((row['Latitude'], row['Longitude']), radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['LaunchSite']))
    # Create a deep orange circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
          (row['Latitude'], row['Longitude']),
          # Create an icon as a text label
          icon=DivIcon(
              icon_size=(20,20),
              icon_anchor=(0,0),
              html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['LaunchSite'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

After exploring the map by zooming-in/out the marked areas, we can  notice that the launch sites are in proximity to the Equator line and are all in very close proximity to the coast. Some of them to the eastern coast and the others are on the western coast.

## Marking the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that dataframe spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not.

In [11]:
spacex_df.tail(10)

,LaunchSite,Latitude,Longitude,Class
80,CCSFS SLC 40,28.561857,-80.577366,1
81,CCSFS SLC 40,28.561857,-80.577366,1
82,CCSFS SLC 40,28.561857,-80.577366,1
83,CCSFS SLC 40,28.561857,-80.577366,1
84,CCSFS SLC 40,28.561857,-80.577366,1
85,KSC LC 39A,28.608059,-80.603956,1
86,KSC LC 39A,28.608059,-80.603956,1
87,KSC LC 39A,28.608059,-80.603956,1
88,CCSFS SLC 40,28.561857,-80.577366,1
89,CCSFS SLC 40,28.561857,-80.577366,1


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed `(class=0)`, we use a red marker.

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()

First, let's create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value.

In [13]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df["marker_color"] = ["green" if classe == 1 else "red" for classe in spacex_df["Class"]]
spacex_df.tail()

,LaunchSite,Latitude,Longitude,Class,marker_color
85,KSC LC 39A,28.608059,-80.603956,1,green
86,KSC LC 39A,28.608059,-80.603956,1,green
87,KSC LC 39A,28.608059,-80.603956,1,green
88,CCSFS SLC 40,28.561857,-80.577366,1,green
89,CCSFS SLC 40,28.561857,-80.577366,1,green


Then, for each launch result in `spacex_df` dataframe, let's add a `folium.Marker` to `marker_cluster`.

In [14]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df dataframe
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch has succeded or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    
    # Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.map.Marker(
          (record['Latitude'], record['Longitude']),
          # Create an icon as a text label
          icon=DivIcon(
              icon_size=(20,20),
              icon_anchor=(0,0),
              html='<div style="font-size: 12; color:' + record["marker_color"] + ';"><b>%s</b></div>' % record['LaunchSite'],
        )
    )
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates (KSC LC 39A).

## Calculating the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railways, etc).

In [15]:
# Add Mouse Position to get the coordinate (Latitude, Longitude) for a mouse over on the map

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now we zoom in to a launch site and explore its proximities to see if we can easily find any railway, highway, coastline, etc. We move the mouse to these points and we mark down their coordinates (shown on the top-left) in order to the distance of the launch site.

In [16]:
# Define a function that calculates the distance between two places or spots on the map giving their coordinates

from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [17]:
# find coordinates of the closet coastline using MousePosition

launch_site_lat = 28.56230197  # Latitude of CCSFS SLC 40 launch site
launch_site_lon = -80.577356  # Longitude of CCSFS SLC 40 launch site
coastline_lat = 28.56367
coastline_lon = -80.57163
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [18]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 

distance_marker = folium.Marker(
    (coastline_lat, coastline_lon),
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map

In [19]:
# Create a `folium.PolyLine` (literally a line) object between the coastline and launch site (CCSFS SLC 40)

lines = folium.PolyLine(locations=((coastline_lat, coastline_lon), (launch_site_lat, launch_site_lon)), weight=1)
site_map.add_child(lines)

Similarly, we can draw a line between a launch site to its closest city, railway, highway, etc. We need to use `MousePosition` to find their coordinates on the map first.

A railway map symbol may look like this :

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this :

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this :

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>

In [20]:
# Create a marker with distance to the closest highway (Samuel C Phillips Parkway) to CCSFS SLC 40 launch site
# Draw a line between the marker to the launch site

lines=folium.PolyLine(locations=((28.56245, -80.57064), (launch_site_lat, launch_site_lon)), weight=1)
site_map.add_child(lines)

We can notice now that CCSFS SLC 40 launch site is close to the Samuel C Phillips Parkway (It's a highway).